# Part Combination Space Exploration

This is a notebook developed to leverage the new SysML v2 semantics for nested features and instantiation of models to generate instances of M1 system models as feedstock for analysis pipelines.

In [ ]:
import requests
import getpass
import math

from __future__ import print_function

import time
import sysml_v2_api_client
from sysml_v2_api_client.rest import ApiException
from pprint import pprint

import json
import networkx as NX
import matplotlib.pyplot as plt

import random
import copy

import pymbe.api as pm
from pymbe.model_loading import ModelingSession as Session
from pymbe.interpretation.interpretation import RandomGenerationStrategy as RGS

In [ ]:
import pymbe.query as pmQuery

# Configure API Server Connection

In [ ]:
sysml_api_base_url = 'http://sysml2-sst.intercax.com:9000'

## Activate APIs

Connect the API classes to the server

In [ ]:
configuration = sysml_v2_api_client.Configuration(
    host = sysml_api_base_url
)

projects_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    project_api_instance = sysml_v2_api_client.ProjectApi(api_client)
    
commits_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    commits_api_instance = sysml_v2_api_client.CommitApi(api_client)
    
elements_api_instance = None

with sysml_v2_api_client.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    elements_api_instance = sysml_v2_api_client.ElementApi(api_client)

## Pull down commits and elements catalogs

With the API handles, use the pre-built methods to get lists of commits and elements.

In [ ]:
project_api_instance

In [ ]:
kerbal_proj = [my_proj for my_proj in project_api_instance.get_projects() if my_proj.name.find('Kerbal') > -1][0]
kerbal_proj

In [ ]:
try:
    # Get commits by project
    commits_response = commits_api_instance.get_commits_by_project(kerbal_proj.id)
    pprint(commits_response)
except ApiException as e:
    print("Exception when calling CommitApi->get_commits_by_project: %s\n" % e)

In [ ]:
elements = []

try:
    # Get commits by project
    elements = elements_api_instance.get_elements_by_project_commit(kerbal_proj.id, commits_response[0].id)
except ApiException as e:
    print("Exception when calling ElementApi->get_elements_by_project_commit: %s\n" % e)

In [ ]:
len(elements)

In [ ]:
pprint(elements[0])

# Gather Element Data

Since the generated API doesn't have much detail for elements, need to do this more hands-on.

Not elegant below to just have a larger page size, should implement paging later on.

In [ ]:
elements_url = (sysml_api_base_url +
                '/projects/{0}/commits/{1}/elements?page[size]=2000').format(kerbal_proj.id, commits_response[0].id)

In [ ]:
elements_url

In [ ]:
elements_response = requests.get(
    elements_url
)

In [ ]:
elements_data = elements_response.json()

Split the elements into relationships and non-relationships. This will let us work with graph representations and a graph understanding of the underlying model.

In [ ]:
len(elements_data)

In [ ]:
non_relations = [non_relation for non_relation in elements_data if not 'relatedElement' in non_relation]

In [ ]:
relations = [relation for relation in elements_data if 'relatedElement' in relation]

In [ ]:
len(non_relations)

In [ ]:
len(relations)

Survey which and how many metatypes are in the model.

In [ ]:
metatypes = []
for nr in elements_data:
    if nr['@type'] not in metatypes:
        metatypes.append(nr['@type'])
        
metatypes

Create a working session for the model and feed it the serialized data. The working session will also generate useful graphs to inspect later in this workbook.

In [ ]:
working_model = Session()
working_model.thaw_json_data(elements_data)

## Show Computed Graphs

### Superclassing Graph

In [ ]:
super_labels = NX.get_node_attributes(working_model.graph_manager.superclassing_graph,'name')

plt.figure(figsize=(20, 12))

NX.draw_planar(working_model.graph_manager.superclassing_graph,
               labels=super_labels)

### Banded Attribute Graph

In [ ]:
list(working_model.graph_manager.banded_featuring_graph.successors('f5f406e8-b8a6-4f8b-a90e-01616a6cf1c1'))

In [ ]:
banded_labels = NX.get_node_attributes(working_model.graph_manager.banded_featuring_graph,'name')

edge_kinds = NX.get_edge_attributes(working_model.graph_manager.banded_featuring_graph,'kind')
colors = {}

for key, value in edge_kinds.items():
    if value == 'Superclassing':
        colors.update({key: 'b'})
    elif value == 'FeatureTyping^-1':
        colors.update({key: 'g'})
    elif value == 'FeatureMembership^-1':
        colors.update({key: 'r'})

plt.figure(figsize=(20, 12))

NX.draw_planar(working_model.graph_manager.banded_featuring_graph,
               labels=banded_labels,
              edge_color=colors.values(),
              arrowsize=20)

I think what we want is to see the chain of superclasses from a feature's owner and compare that to the path through redefinition to classifiers.

[
    edge
    for edge in lpg.graph.edges
    if edge[2] in ("ReturnParameterMembership", "Superclassing")
]

In [ ]:
att_uses = working_model.get_all_of_metaclass(metaclass_name="AttributeUsage")

## Inspect Part Usages

In [ ]:
part_uses = working_model.get_all_of_metaclass(metaclass_name='PartUsage')

In [ ]:
len(part_uses)

In [ ]:
['{0} has multiplicity {1}..{2}'.format(
    part_use['name'],
    working_model.feature_lower_multiplicity(part_use['@id']),
    working_model.feature_upper_multiplicity(part_use['@id'])
) for part_use in part_uses]

Get feature types.

In [ ]:
feature_types = [feature_type for feature_type in relations if feature_type['@type'] == 'FeatureTyping']

# Generate Instances

With the base semantic model in hand, begin to apply the rules to generate our system alternatives.

## Find number of instances for feature last positions

In SysML, the default type is PartDefinition, which is a Classifier, meaning the minimal interpretation of length one (the specific instance). Nesting parts then have an interpretation as expected by systems engineers, namely that the instances "stack" in order to provide a navigation from top-level assembly to leaf component.

In [ ]:
['{0} needs {1} instances of type {2}'.format(
    part_use['name'],
    working_model.feature_upper_multiplicity(part_use['@id']),
    working_model.graph_manager.get_feature_type_name(part_use['@id'])
) for part_use in part_uses]

Automatically shorten names so that sequences remain readable when printed.

In [ ]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

## Determine the size of the universe of instances needed for creating alternatives

Use feature membership together with multiplicity to decide how many individuals are needed.

In [ ]:
working_model.graph_manager.roll_up_upper_multiplicities()

In [ ]:
[(working_model.get_name_by_id(key), mult) for key, mult in
    working_model.graph_manager.roll_up_upper_multiplicities().items()
]

### Subdivide Abstract Feature Types

Look at the feature types for where they are abstract and then generate instances from the more specific types.

In [ ]:
abstract_types = []
for typ in working_model.graph_manager.superclassing_graph.nodes():
    ele_data = working_model.get_data_by_id(ele_id=typ)
    if ele_data['isAbstract']:
        abstract_types.append(ele_data)
        
print([abstract['name'] for abstract in abstract_types])

In [ ]:
[working_model.get_name_by_id(definite)
 for definite in working_model.graph_manager.partition_abstract_type(abstract_type_id=abstract_types[3]['@id'])]

## Generate Random Alternatives

Start creating the alternatives with random draws on multiplicity. This will be our space for investigation for weights, thrust-to-weight ratios at stage ignitions, delta-Vs, and initial and burnout masses.

In [ ]:
working_model.get_name_by_id(ele_id='000f78ae-e890-4f81-847c-f9bf976abe1b')

In [ ]:
generator = RGS(number_of_cases=2, model_session=working_model, short_names=shorten_pre_bake)

In [ ]:
generator.pprint_attribute_solution(solution_no=0)

In [ ]:
generator.pprint_feature_solution(solution_no=0)

### Look at attribute values and feature types 

Build a graph where attribute usages are connected to other types and superclassing

In [ ]:
for ft in working_model.get_all_of_metaclass(metaclass_name='FeatureMembership'):
    print(working_model.get_element_signature(ft))

## Investigate Literal Assignments and Expressions

In [ ]:
expressions = [expr for expr in non_relations if 'Expression' in expr['@type']]

In [ ]:
len(expressions)

In [ ]:
def expression_signature(expr):
    if expr['@type'] == 'FeatureReferenceExpression':
        fre = working_model.get_data_by_id(expr['referent']['@id'])
        return working_model.get_name_by_id(fre['owner']['@id']) + '::' + fre['name'] + '(ref)'
    elif expr['@type'] == 'OperatorExpression':
        oper = expr['operator']
        paras = [expression_signature(working_model.get_data_by_id(para['@id'])) for para in expr['operand']]
        return str(oper) + ' (' + str(paras) + ')'
    elif expr['@type'] == 'Expression':
        result_name = working_model.get_name_by_id(expr['result']['@id'])
        inputs = [expression_signature(working_model.get_data_by_id(para['@id'])) for para in expr['input']]
        return 'f(' + ', '.join(inputs) + ') => ' + result_name
    elif expr['@type'] in ('ReferenceUsage', 'Feature', 'AttributeUsage', 'Function'):
        if expr['name'] is not None:
            return expr['name']
        else:
            return ''
    elif expr['@type'] == 'InvocationExpression':
        invoked = working_model.get_name_by_id(expr['type'][0]['@id'])
        inputs = [expression_signature(working_model.get_data_by_id(para['@id'])) for para in expr['input']]
        return invoked + '(' + str(inputs) + ')'
    else:
        return expr['@type']

In [ ]:
expression_signature(expressions[9])

In [ ]:
expression_graph = NX.DiGraph()

In [ ]:
for rel in relations:
    if rel['@type'] == 'ReturnParameterMembership':
        owner = working_model.get_data_by_id(rel['owningType']['@id'])
        para = working_model.get_data_by_id(rel['memberParameter']['@id'])
        
        expression_graph.add_node(owner['@id'], label=expression_signature(owner))
        expression_graph.add_node(para['@id'], label=expression_signature(para))
        expression_graph.add_edge(owner['@id'], para['@id'], kind='ReturnParameterMembership')
        
        print (expression_signature(owner) + ' =RPN=> ' + expression_signature(para))
    elif rel['@type'] == 'ParameterMembership':
        owner = working_model.get_data_by_id(rel['owningType']['@id'])
        para = working_model.get_data_by_id(rel['memberParameter']['@id'])
        
        expression_graph.add_node(owner['@id'], label=expression_signature(owner))
        expression_graph.add_node(para['@id'], label=expression_signature(para))
        expression_graph.add_edge(owner['@id'], para['@id'], kind='ParameterMembership')
        print (expression_signature(owner) + ' =PN=> ' + expression_signature(para))
    elif rel['@type'] == 'FeatureValue':
        feature = working_model.get_data_by_id(rel['owningRelatedElement']['@id'])
        value = working_model.get_data_by_id(rel['value']['@id'])
        
        expression_graph.add_node(feature['@id'], label=expression_signature(feature))
        expression_graph.add_node(value['@id'], label=expression_signature(value))
        expression_graph.add_edge(feature['@id'], value['@id'], kind='FeatureValue')
        
        feature_owner = working_model.get_data_by_id(feature['owningType']['@id'])
        
        if 'value' in value:
            print (expression_signature(value) + ' (' + str(value['value']) + \
                   ') value of ' + feature_owner['name'] + '::' + expression_signature(feature))
        else:
            print (expression_signature(value) + \
                   ' value of ' + feature_owner['name'] + '::' + expression_signature(feature))
            
    elif rel['@type'] == 'FeatureMembership':
        if 'Expression' in working_model.get_data_by_id(rel['memberFeature']['@id'])['@type']:
            if working_model.get_metaclass_by_id(rel['memberFeature']['@id']) != 'AttributeUsage':
                owner = working_model.get_data_by_id(rel['owningType']['@id'])
                expr = working_model.get_data_by_id(rel['memberFeature']['@id'])

                expression_graph.add_node(owner['@id'], label=expression_signature(owner))
                expression_graph.add_node(expr['@id'], label=expression_signature(expr))
                expression_graph.add_edge(owner['@id'], expr['@id'], kind='FeatureMembership')
            
            print (expression_signature(owner) + ' =FM=> ' + expression_signature(expr))
            
    elif rel['@type'] == 'ResultExpressionMembership':
        if 'Expression' in working_model.get_data_by_id(rel['memberFeature']['@id'])['@type']:
            if working_model.get_metaclass_by_id(rel['memberFeature']['@id']) != 'AttributeUsage':
                owner = working_model.get_data_by_id(rel['owningType']['@id'])
                expr = working_model.get_data_by_id(rel['memberFeature']['@id'])

                expression_graph.add_node(owner['@id'], label=expression_signature(owner))
                expression_graph.add_node(expr['@id'], label=expression_signature(expr))
                expression_graph.add_edge(owner['@id'], expr['@id'], kind='FeatureMembership')
            
            print (expression_signature(owner) + ' =REM=> ' + expression_signature(expr))
            
    elif rel['@type'] == 'FeatureTyping':
        if 'Expression' in working_model.get_data_by_id(rel['typedFeature']['@id'])['@type']:
            typ = working_model.get_data_by_id(rel['type']['@id'])
            expr = working_model.get_data_by_id(rel['typedFeature']['@id'])

            expression_graph.add_node(typ['@id'], label=expression_signature(typ))
            expression_graph.add_node(expr['@id'], label=expression_signature(expr))
            expression_graph.add_edge(expr['@id'], typ['@id'], kind='FeatureTyping')
            
            print (expression_signature(expr) + ' =FT=> ' + expression_signature(typ))

In [ ]:
expression_value_pairs = []

for rel in relations:
    if rel['@type'] == 'FeatureValue':
        value = working_model.get_data_by_id(rel['value']['@id'])
        if 'Expression' in value['@type']:
            feature = working_model.get_data_by_id(rel['owningRelatedElement']['@id'])
            expression_value_pairs.append([feature['@id'], value['@id']])

In [ ]:
len(expression_value_pairs)

In [ ]:
def get_context_from_graph(graph, expr_values):
    context_dict = {}
    for ev in expr_values:
        tree = NX.dfs_tree(graph, source=ev[1])
        for tree_node in tree.nodes():
            if 'Expression' in working_model.get_data_by_id(tree_node)['@type']:
                context_dict.update({tree_node: ev[0]})
                
    return context_dict

In [ ]:
context_dict = get_context_from_graph(expression_graph, expression_value_pairs)
context_dict

Examine the expression graph and paint expression nodes with context for later evaluations.

In [ ]:
for comp in NX.connected_components(expression_graph.to_undirected()):
    connected_sub = NX.subgraph(expression_graph, list(comp))
    
    pos = NX.planar_layout(connected_sub)

    banded_labels = NX.get_node_attributes(connected_sub,'label')

    edge_kinds = NX.get_edge_attributes(connected_sub,'kind')
    colors = []
    
    rpm_edges = []
    pm_edges = []
    fv_edges = []
    ft_edges = []
    fm_edges = []
    rem_edges = []
    
    for edg in connected_sub.edges.data("kind"):
        if edg[2] == 'ReturnParameterMembership':
            rpm_edges.append([edg[0], edg[1]])
        elif edg[2] == 'ParameterMembership':
            pm_edges.append([edg[0], edg[1]])
        elif edg[2] == 'FeatureValue':
            fv_edges.append([edg[0], edg[1]])
        elif edg[2] == 'FeatureTyping':
            ft_edges.append([edg[0], edg[1]])
        elif edg[2] == 'FeatureMembership':
            fm_edges.append([edg[0], edg[1]])
        elif edg[2] == 'ResultExpressionMembership':
            rem_edges.append([edg[0], edg[1]])
    
    # only plot non-trivial
    if len(comp) > 4:

        plt.figure(figsize=(20, 12))
        
        NX.draw_networkx_nodes(connected_sub, pos, nodelist=comp, node_color="r")
        
        NX.draw_networkx_edges(connected_sub, pos, edgelist=rpm_edges, edge_color="b", arrowsize=20)
        #NX.draw_networkx_edges(connected_sub, pos, edgelist=ft_edges, edge_color="k", arrowsize=20)
        NX.draw_networkx_edges(connected_sub, pos, edgelist=pm_edges, edge_color="g", arrowsize=20)
        #NX.draw_networkx_edges(connected_sub, pos, edgelist=fv_edges, edge_color="r", arrowsize=20)
        #NX.draw_networkx_edges(connected_sub, pos, edgelist=fm_edges, edge_color="c", arrowsize=20)
        NX.draw_networkx_edges(connected_sub, pos, edgelist=rem_edges, edge_color="m", arrowsize=20)
        
        NX.draw_networkx_labels(connected_sub, pos, banded_labels, font_size=12)

        #NX.draw_planar(connected_sub,
        #               edgelist=connected_sub.edges,
        #               labels=banded_labels,
        #              edge_color=colors,
        #              arrowsize=20)

In [ ]:
generator.attribute_dicts[0]['25a4a485-735c-4b5d-8d6f-140e176a144f']

In [ ]:
collects = [nr for nr in non_relations if nr['@type'] == 'OperatorExpression' and nr['operator'] == 'collect']

In [ ]:
len(collects)

In [ ]:
def filter_feature_by_context(context, feature_list):
    filtered = [feat for feat in feature_list if str(context) == str(feat[0:len(context)])]
    return filtered

Simpler answer: The dot operator on instances.

In [ ]:
generator.classifier_instance_dicts[0]['f25193df-abf0-478b-997d-dbee3895090e']

In [ ]:
generator.feature_instance_dicts[0]['cd081fb5-d4b7-42bc-880b-9f4cf2517394']

In [ ]:
def sequence_dot_operator(left_item, right_side_seqs):
    left_len = len(left_item)
    right_len = len(right_side_seqs[0])
    #print('Left is ' + str(left_len) + ' right is ' + str(right_len))
    matched_items = []
    
    for right_item in right_side_seqs:
        #print(str(right_item[0:(right_len-1)]))
        if left_len != right_len:
            if str(left_item) == str(right_item[0:(right_len-1)]):
                matched_items.append(right_item)
        else:
            if str(left_item[1:None]) == str(right_item[0:(right_len-1)]):
                matched_items.append(right_item)
        
    return matched_items

In [ ]:
step_1 = sequence_dot_operator(generator.classifier_instance_dicts[0]['f25193df-abf0-478b-997d-dbee3895090e'][0],
                     generator.feature_instance_dicts[0]['cd081fb5-d4b7-42bc-880b-9f4cf2517394'])

In [ ]:
step_1

In [ ]:
all_masses = [
    sequence_dot_operator(step_1_step,
                     generator.attribute_dicts[1]['6045bd07-09d9-4a01-97a4-ede5cad64d76'])
    for step_1_step in step_1
]

In [ ]:
all_masses
sum(all_masses, [])

In [ ]:
sequence_dot_operator(step_1[2],
                     generator.attribute_dicts[1]['6045bd07-09d9-4a01-97a4-ede5cad64d76'])

In [ ]:
sequence_dot_operator(generator.classifier_instance_dicts[0]['f25193df-abf0-478b-997d-dbee3895090e'],
                     generator.feature_instance_dicts[0]['000f78ae-e890-4f81-847c-f9bf976abe1b'])

In [ ]:
sequence_dot_operator(generator.feature_instance_dicts[0]['000f78ae-e890-4f81-847c-f9bf976abe1b'],
                     generator.attribute_dicts[0]['f912dc27-fc5e-48e8-abf7-29d3f47c913c'])